In [19]:
import sys
import pickle
import numpy as np
import pandas as pd
#from nlp_utils import clean_text, pos_tag_text
sys.path.append("../")
from param_config import config
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import xgboost as xgb
import datetime

pd.set_option('display.float_format',lambda x: '%.5f'%x)


In [20]:
dfAll = pd.read_csv('../../Data/All_Data_dedup.csv')

In [ ]:
dfAll2 = pd.read_csv('../../Data/All_Data_dedup.csv')

In [36]:
for var in dfAll.columns:
    if 'TOOL' in var:
        tmpTrain = pd.get_dummies(dfAll[var],prefix=var,dummy_na=True)
        dfAll = pd.concat([dfAll,tmpTrain],axis=1)
        del dfAll[var]

In [37]:
tmp = dfAll.isnull().sum().to_dict()
null_dict ={}
for key,value in tmp.items():
    if key=='ID':
        continue
    if not null_dict.__contains__(value):
        null_dict[value] = [key]
    else:
        null_dict[value].append(key)

In [38]:
dfAll_input = dfAll[null_dict[0]]
for key,value in null_dict.items():
    if key==0:
        continue
    for var in value:
        train = dfAll_input.loc[dfAll[var].notnull()]
        target = dfAll.loc[dfAll[var].notnull(),var]
        test = dfAll_input.loc[dfAll[var].isnull()]
        model = RandomForestRegressor(n_estimators=1999, max_features='sqrt',  max_depth=4, random_state=615)  
        model.fit(train,target)
        predicted = model.predict(test)
        dfAll.loc[dfAll[var].isnull(),var] = predicted
        print(var)

220X25
220X26
220X27
220X28
220X29
220X30
220X31
220X32
220X33
220X34
220X35
220X36
220X37
220X38
220X39
220X40
220X53
220X54
220X55
220X67
220X107
220X108
220X222
220X223
220X224
220X225
220X241
220X247
220X248
220X249
220X260
220X262
220X266
220X269
220X270
220X271
220X272
220X273
220X274
220X275
220X277
220X279
220X281
220X282
220X284
220X286
220X288
220X289
220X290
220X291
220X293
220X294
220X295
220X296
220X297
220X298
220X299
220X300
220X301
220X302
220X303
220X304
220X305
220X306
220X307
220X308
220X309
220X310
220X311
220X312
220X313
220X314
220X315
220X316
220X317
220X318
220X321
220X322
220X323
220X324
220X325
220X326
220X327
220X328
220X329
220X330
220X331
220X332
220X333
220X334
220X335
220X336
220X337
220X338
220X339
220X340
220X341
220X342
220X343
220X344
220X345
220X346
220X347
220X348
220X349
220X350
220X351
220X352
220X353
220X354
220X355


KeyboardInterrupt: 

In [2]:
print("Load data...")

dfTrain = pd.read_csv(config.original_train_data_path)

dfPred= pd.read_csv(config.original_test_data_path)
dfPred2= pd.read_csv(config.original_test_data2_path)
# number of train/test samples
num_train, num_pred = dfTrain.shape[0], dfPred.shape[0]

print("Done.")

f = open('../../Data/col_name.txt','r')  
a = f.read()  
col_name = eval(a)  
f.close() 

f = open('../../Data/Procedure.txt','r')  
a = f.read()  
procedure = eval(a)  
f.close()

f = open('../../Cache/var_change.txt','r')
a = f.read()
var_change = eval(a)
f.close()

dfTrain = dfTrain.rename(columns=col_name)
raw_predictors = dfTrain.columns.tolist()[1:-1]
dfTrain = dfTrain.set_index(dfTrain['ID'])
dfPred = dfPred.rename(columns=col_name)
dfPred = dfPred.set_index(dfPred['ID'])
dfPred2 = dfPred2.rename(columns=col_name)

dfPredAll = pd.concat([dfPred,dfPred2],axis=0)
dfPredAll = dfPredAll.set_index(dfPredAll['ID'])

dfAll = pd.concat([dfTrain,dfPred,dfPred2],axis=0)

Load data...
Done.


In [3]:
#load features
Feat_file = ['Feat_cnt_col','Feat_cnt_tool','Feat_pcent_col','Feat_pcent_tool']
ex_feat = []
for filename in Feat_file:
    tmp = pd.read_csv('../../Cache/'+filename+'.csv')
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    ex_feat += tmp_feat_list
    dfTrain = pd.merge(dfTrain,tmp,'left','ID')
    dfPred = pd.merge(dfPred,tmp,'left','ID')
   
        


In [ ]:
##define the variables
'''var_change = {'raw':raw}
for filename in Feat_file:
    tmp = pd.read_csv('../../Cache/'+filename+'.csv',nrows=10)
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    var_change[filename] = {'constant':[],'category':[]}
    for var in tmp_feat_list:
        if dfTrain[var].nunique()==1 and dfTrain[var].isnull().sum()==0:
            var_change[filename]['constant'].append(var)
        elif dfTrain[var].nunique()<=5 or 'TOOL' in var:
            var_change[filename]['category'].append(var)
'''



In [4]:
#define the input variables, dummies
predictors = raw_predictors+ex_feat
feat_cnt = 0
for key,value in var_change.items():
    feat_cnt+=1
    if feat_cnt>1:
        continue
    for var in var_change[key]['constant']:
        predictors.remove(var)
    for var in var_change[key]['category']:
        predictors.remove(var)
        tmpTrain = pd.get_dummies(dfTrain[var],prefix=var,dummy_na=True)
        tmpPred = pd.get_dummies(dfPred[var],prefix=var,dummy_na=True)
        predictors = predictors + tmpTrain.columns.tolist()
        dfTrain = pd.concat([dfTrain,tmpTrain],axis=1)
        dfPred = pd.concat([dfPred,tmpPred],axis=1)
for var in predictors:
    if var not in dfPred.columns:
        dfPred[var] = 0

In [26]:
def xgb_kfold(dfTrain,dfPred,predictors,n_splits=5, params = {'max_depth':4, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':2,'subsample':0.8,
                         'colsample_bytree':0.8}):  
    kf = KFold(n_splits=n_splits,shuffle=True)
    dpred = xgb.DMatrix(dfPred[predictors].values,label=[0]*len(dfPred),missing=np.nan)
    imp = pd.DataFrame({'variable':predictors,'lk':['f'+str(i) for i in range(len(predictors))]})
    round=0
    for train_index, test_index in kf.split(dfTrain):
        round+=1
        train_X = dfTrain.loc[train_index,predictors]
        test_X = dfTrain.loc[test_index,predictors]
        train_Y = dfTrain.loc[train_index,'Y']
        test_Y = dfTrain.loc[test_index,'Y']

        dtrain = xgb.DMatrix(train_X.values, label=train_Y.values, missing = np.nan)
        dtest = xgb.DMatrix(test_X.values, label=test_Y.values, missing = np.nan)
        param = params 
        evallist  = [(dtrain,'train'),(dtest,'eval')]  
        num_round = 5000
        evals_dict = {}
        model = xgb.train(param,dtrain,num_round, evallist,early_stopping_rounds=50,evals_result=evals_dict,verbose_eval =1000)
        performance_df = pd.DataFrame(evals_dict['eval'])
        bst_tree = len(performance_df)-51
        pred_test = model.predict(dtest,ntree_limit =bst_tree)

        tmp_imp = pd.DataFrame(model.get_score(importance_type='gain'),index=['imp_fold%d'%round]).T
        tmp_imp['lk'] = tmp_imp.index
        imp = imp.merge(tmp_imp,'left','lk').fillna(0)


        pred_score = model.predict(dpred,ntree_limit =bst_tree)
        if round==1:
            test_result = pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})
            result = pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score})
        else:
            test_result = pd.concat([test_result,pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})],axis=0)
            result = result.merge(pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score}),'inner','ID')
    print("Test MSE:",metrics.mean_squared_error(test_result['target'], test_result['score']))
    return test_result,result,imp,metrics.mean_squared_error(test_result['target'], test_result['score'])

In [ ]:
record = []
for md in [3,4,5]:
    for la in [0.8,1,1.3,1.5,2]:
        for sp in [0.6,0.8,1]:
            for cb in [0.2,0.4,0.6,0.8,1]:
                tmp_record = [md,la,sp,cb]
                print(tmp_record)
                params = {'max_depth':md, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':la,'subsample':sp,'colsample_bytree':cb}
                tmp_record.append(xgb_kfold(dfTrain,dfPred,predictors,5,params=params)[3])
                record.append(tmp_record)
                GS_df = pd.DataFrame(record,columns=['max_depth','lambda','subsample','colsample_bytree','test_MSE'])
                GS_df.to_csv('../../Profiling/GS_result.csv',index=False)

[3, 0.8, 0.6, 0.2]
[0]	train-rmse:2.33435	eval-rmse:2.31993
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[508]	train-rmse:0.080587	eval-rmse:0.17733

[0]	train-rmse:2.32603	eval-rmse:2.35314
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[1000]	train-rmse:0.036267	eval-rmse:0.190875
Stopping. Best iteration:
[964]	train-rmse:0.038393	eval-rmse:0.19084

[0]	train-rmse:2.3309	eval-rmse:2.33339
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[626]	train-rmse:0.067672	eval-rmse:0.14085

[0]	train-rmse:2.32928	eval-rmse:2.34094
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved i

In [12]:
test_result,result,imp = xgb_kfold(dfTrain,dfPred,predictors)

[0]	train-rmse:2.33325	eval-rmse:2.32446
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[50]	train-rmse:1.42504	eval-rmse:1.41672
[100]	train-rmse:0.879138	eval-rmse:0.872982
[150]	train-rmse:0.55208	eval-rmse:0.555279
[200]	train-rmse:0.355857	eval-rmse:0.372674
[250]	train-rmse:0.236805	eval-rmse:0.272561
[300]	train-rmse:0.163235	eval-rmse:0.220306
[350]	train-rmse:0.117279	eval-rmse:0.19572
[400]	train-rmse:0.088505	eval-rmse:0.184687
[450]	train-rmse:0.069554	eval-rmse:0.180748
[500]	train-rmse:0.056817	eval-rmse:0.178809
[550]	train-rmse:0.047473	eval-rmse:0.17801
[600]	train-rmse:0.04041	eval-rmse:0.177439
[650]	train-rmse:0.034622	eval-rmse:0.17706
[700]	train-rmse:0.029927	eval-rmse:0.177028
[750]	train-rmse:0.025907	eval-rmse:0.176934
[800]	train-rmse:0.022595	eval-rmse:0.176859
[850]	train-rmse:0.019805	eval-rmse:0.176851
Stopping. Best iteration:
[819]	train-rmse:0.021476	eval-rm

In [13]:
imp.to_csv('../../Profiling/imp_test_v3.csv',index=False)

In [ ]:
result['score']=result[['Score_1','Score_2','Score_3','Score_4','Score_5']].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
submit.to_csv('../../Submission/submit_%s.csv'%today,header=False,index=False)

In [ ]:
#貌似RF根本就不需要CV啊，其实也行吧
def rf_kfold(dfTrain,dfPred,predictors,n_splits=5):  
    kf = KFold(n_splits=n_splits,shuffle=True)
    dpred = xgb.DMatrix(dfPred[predictors].values,label=[0]*len(dfPred),missing=np.nan)
    imp = pd.DataFrame({'variable':predictors,'lk':['f'+str(i) for i in range(len(predictors))]})
    round=0
    for train_index, test_index in kf.split(dfTrain):
        round+=1
        train_X = dfTrain.loc[train_index,predictors].values
        test_X = dfTrain.loc[test_index,predictors].values
        train_Y = dfTrain.loc[train_index,'Y'].values
        test_Y = dfTrain.loc[test_index,'Y'].values
 
        model = RandomForestRegressor(n_estimators=1999, max_features='sqrt', max_depth=None, max_depth=4, compute_importances=True, random_state=615)  
        model.fit(train_X,train_Y)
        pred_test = model.predict(test_X）
        pred_score = model.predict(dfPred[predictors].values)
        if round==1:
            test_result = pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})
            result = pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score})
        else:
            test_result = pd.concat([test_result,pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})],axis=0)
            result = result.merge(pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score}),'inner','ID')
    print("Test MSE:",metrics.mean_squared_error(test_result['target'], test_result['score']))
    return test_result,result,imp,metrics.mean_squared_error(test_result['target'], test_result['score'])

In [ ]:
#RF填缺失值